# Address parsing and correction

In this notebook, I will describe the use of CRF models in [libpostal address parser](https://github.com/openvenues/libpostal) and how they differ from newer language processing methods, namely RNN and LSTM. In the CRF model each sequence is remembered and a tag is given to each word, depending on the neighbouring words.  The neighbouring words are kept in a structure which can be represented as a vector:

![one-hot encodings](assets/ccrfDiagram.png)

The main limitation here is in the vector that keeps the neighbourhood information. The model will learn to associate words to tags over time, however only in that fixed relative position with each other. If there are additional words to change the structure of an input, as it was during training, the output will not be reproducible. As an example lets take a model that has been trained with input of this format:

64 Great Eastern Street London EC2A 3QR

If we now insert new information about the whereabouts of our address into the input:

64 Great Eastern Street Ingrove Court London EC2A 3QR

 Assume such an address exists but Ingrove Court does not need to be included for the postman to identify this address. The model cannot tell you whether Ingrove is part of the street. It has never seen data before to show it that explicitly Ingrove Court is an optional part of the address. This is why it is a good idea to find common points in the data where there can be additional words with different contextual meaning eg. street names, suburbs, cities etc etc... Commonly in NLP only the most common words are used to represent meaning of a sentence or tagging of word meanings.

An LSTM in contrast, is capable of remembering word occurances that are several words away and will. This is by virtue of a non-linearity in nearby word association that is not simply composed of a vector, rather it is a state represented by neural networks.

In order to demonstrate this associations I will create word embeddings for a small set of addresses. I'll convert addresses to vectors through TensorFlow and understand the common meaning behind them.


Embeddings are just a single layer neural network. We call this layer the embedding layer and the weights are embedding weights. We skip the multiplication into the embedding layer by instead directly grabbing the hidden layer values from the weight matrix. 

![lookup](assets/lookup_matrix.png)




In [1]:
import time

import numpy as np
import tensorflow as tf

import utils

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Load the [openaddress US North East dataset](https://s3.amazonaws.com/data.openaddresses.io/openaddr-collected-us_northeast.zip), and extract onto 'openaddr' directory if not found. read the csv files and load address dictionaries

In [2]:
import glob
import os
import pandas as pd

from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
import zipfile

dataset_folder_path = 'openaddr'
dataset_filename = 'openaddr-collected-us_northeast.zip'
dataset_name = 'Openaddress Dataset'

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile(dataset_filename):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc=dataset_name) as pbar:
        urlretrieve(
            'https://s3.amazonaws.com/data.openaddresses.io/openaddr-collected-us_northeast.zip',
            dataset_filename,
            pbar.hook)

if not isdir(dataset_folder_path):
    with zipfile.ZipFile(dataset_filename) as zip_ref:
        zip_ref.extractall(dataset_folder_path)


id_to_address = {}
address_to_id = {}
i = 0
for state in os.listdir('./openaddr/us'):
    
    for filename in glob.glob('./openaddr/us/{}/*.csv'.format(state)):
        csv = pd.read_csv(filename)
        if i == 0:
            print("Example data:\n {}".format(csv.iloc[2]))

        stack = np.stack((csv['CITY'],), axis=-1)
        for j in stack:
            addr = " ".join([str(k).lower()
                             for k in j if not isinstance(k, type(np.nan))])
            if addr not in address_to_id and addr != '' and addr != ' ':
                id_to_address[i] = addr
                address_to_id[addr] = i
                i += 1

del csv

/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Example data:
 LON                 -69.7958
LAT                  44.6535
NUMBER                  1033
STREET             E Pond Rd
UNIT                     NaN
CITY                     NaN
DISTRICT            Somerset
REGION                    ME
POSTCODE                4978
ID                       NaN
HASH        8bbbfb5152ee2563
Name: 2, dtype: object
Example data:
 LON                   -80.0287
LAT                    40.7328
NUMBER                    1260
STREET      MARS-EVANS CITY RD
UNIT                       NaN
CITY                 ADAMS TWP
DISTRICT                   NaN
REGION                     NaN
POSTCODE                   NaN
ID                         NaN
HASH          1b447375afa847e4
Name: 2, dtype: object


/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ubuntu/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (3,4,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler

## Preprocessing

The addresses have all been converted to lower case and only city names are being used and tested for correction cases. vocabulary is the letters from which the city is made up of. We vectorise by counting the number of letters and normalizing to the total number between zero and one. Visualisations below.

In [3]:
print([i for i in address_to_id.keys()][:30])

['adams twp', 'cranberry twp', 'middlesex twp', 'mars boro', 'allegheny twp', 'bruin boro', 'brady twp', 'slippery rock twp', 'buffalo twp', 'winfield twp', 'butler city', 'butler twp', 'penn twp', 'callery boro', 'center twp', 'clay twp', 'cherry twp', 'cherry valley boro', 'chicora boro', 'clearfield twp', 'donegal twp', 'summit twp', 'clinton twp', 'connoq boro', 'forward twp', 'connoq twp', 'lancaster twp', 'seven fields boro', 'fairview twp', 'east butler boro']


In [4]:
print("Total addresses: {}".format(len(address_to_id)))

vocab_to_id = {}
int_to_vocab = {}
idx = 0
for address in address_to_id.keys():
    for j in range(len(address)):
        if address[j] not in vocab_to_id:
            vocab_to_id[address[j]] = idx
            int_to_vocab[idx] = address[j]
            idx += 1

vocab_size = len(vocab_to_id)
print("Total unique letters: {}".format(vocab_size))

Total addresses: 4155
Total unique letters: 45


In [5]:
features = []
labels = []
labels_to_id = {}
count = 0

for address, idx in address_to_id.items():
    X = np.zeros(vocab_size)
    Y = np.zeros(len(address_to_id))
    for j in range(len(address)):
        X[vocab_to_id[address[j]]] += 1 
    
    # normalize input sizes 
    X = ((X - X.min()) / (X.max() - X.min())) + 0.001
    
    features.append(X)
    
    labels_to_id[idx] = count
    Y[idx] = 1
    labels.append(Y)
    count += 1
    
print("Example feature vector and label:")
print(features[500])
print(labels[500])

Example feature vector and label:
[1.001e+00 1.000e-03 1.001e+00 1.001e+00 1.000e-03 1.000e-03 1.000e-03
 1.000e-03 1.000e-03 1.001e+00 1.000e-03 1.000e-03 1.000e-03 1.000e-03
 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03
 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03
 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03
 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03 1.000e-03
 1.000e-03 1.000e-03 1.000e-03]
[0. 0. 0. ... 0. 0. 0.]


In [7]:
# Size of the encoding layer (the hidden layer)
encoding_dim = 500
alpha = 0.1

# Input and target placeholders
inp_shape = vocab_size

inputs_ = tf.placeholder(tf.float32 ,(None, inp_shape))
targets_ = tf.placeholder(tf.float32 ,(None, len(address_to_id)))

# Input layer is 45
x1 = tf.expand_dims(inputs_, axis=-1)
x1 = tf.layers.conv1d(x1, filters=32, kernel_size=5, strides=1, padding='valid')
x1 = tf.nn.dropout(x1, 0.5)
relu1 = tf.maximum(alpha * x1, x1)
print(relu1.shape)
# 41x32

x2 = tf.layers.conv1d(relu1, filters=64, kernel_size=5, strides=1, padding='valid')
x2 = tf.nn.dropout(x2, 0.5)
relu2 = tf.maximum(alpha * x2, x2)
print(relu2.shape)
# 37x64

x3 = tf.layers.conv1d(relu2, filters=128, kernel_size=5, strides=1, padding='valid')
x3 = tf.nn.dropout(x3, 0.5)
relu3 = tf.maximum(alpha * x3, x3)
print(relu3.shape)
# 33x128

# global average pooling layer
first_h_dim = tf.reduce_mean(relu3, axis=2)
print(first_h_dim.shape)

# dense layer
encoded = tf.layers.dense(first_h_dim, encoding_dim)
relu4 = tf.maximum(alpha * encoded, encoded)
print(relu4.shape)

# Output layer logits
logits = tf.layers.dense(relu4, len(address_to_id), activation=None)
print(logits.shape)
# Sigmoid output from logits
#decoded = tf.nn.sigmoid(logits, name='outputs')

# loss = tf.log(tf.reduce_sum(tf.abs(targets_ - logits)) + 1)
loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels=targets_, logits=logits)
# Mean of the loss
cost = tf.reduce_mean(loss)




(?, 41, 32)
(?, 37, 64)
(?, 33, 128)
(?, 33)
(?, 500)
(?, 4155)


In [8]:
# Adam optimizer
opt = tf.train.AdamOptimizer(0.0005).minimize(cost)

# Create the session
sess = tf.Session()

epochs = 500
batch_size = 128
sess.run(tf.global_variables_initializer())
for e in range(epochs):
    for ii in range((len(features)//batch_size) - 1):
        batch = features[ii*batch_size:(ii + 1)*batch_size]
        
        # add random noise
        for jj in batch:
            jj[np.random.randint(vocab_size)] += 0.02
            jj[np.random.randint(vocab_size)] -= 0.02
        
        targ = np.array(labels[ii*batch_size:(ii + 1)*batch_size])

        feed = {inputs_: batch, targets_: targ}
        batch_cost, _ = sess.run([cost, opt], feed_dict=feed)

        print("Epoch: {}/{}...".format(e+1, epochs),
              "Training loss: {:.4f}".format(batch_cost))

Epoch: 1/500... Training loss: 8.3322
Epoch: 1/500... Training loss: 8.3323
Epoch: 1/500... Training loss: 8.3326
Epoch: 1/500... Training loss: 8.3325
Epoch: 1/500... Training loss: 8.3327
Epoch: 1/500... Training loss: 8.3327
Epoch: 1/500... Training loss: 8.3328
Epoch: 1/500... Training loss: 8.3333
Epoch: 1/500... Training loss: 8.3335
Epoch: 1/500... Training loss: 8.3338
Epoch: 1/500... Training loss: 8.3340
Epoch: 1/500... Training loss: 8.3344
Epoch: 1/500... Training loss: 8.3344
Epoch: 1/500... Training loss: 8.3345
Epoch: 1/500... Training loss: 8.3350
Epoch: 1/500... Training loss: 8.3350
Epoch: 1/500... Training loss: 8.3351
Epoch: 1/500... Training loss: 8.3354
Epoch: 1/500... Training loss: 8.3355
Epoch: 1/500... Training loss: 8.3356
Epoch: 1/500... Training loss: 8.3358
Epoch: 1/500... Training loss: 8.3361
Epoch: 1/500... Training loss: 8.3364
Epoch: 1/500... Training loss: 8.3366
Epoch: 1/500... Training loss: 8.3367
Epoch: 1/500... Training loss: 8.3372
Epoch: 1/500

Epoch: 8/500... Training loss: 8.2111
Epoch: 8/500... Training loss: 8.1771
Epoch: 8/500... Training loss: 8.1401
Epoch: 8/500... Training loss: 8.0983
Epoch: 8/500... Training loss: 8.1891
Epoch: 8/500... Training loss: 8.1354
Epoch: 8/500... Training loss: 8.1351
Epoch: 8/500... Training loss: 8.0781
Epoch: 8/500... Training loss: 8.0532
Epoch: 8/500... Training loss: 8.0707
Epoch: 8/500... Training loss: 8.0526
Epoch: 8/500... Training loss: 8.0112
Epoch: 8/500... Training loss: 7.8902
Epoch: 8/500... Training loss: 7.9451
Epoch: 8/500... Training loss: 7.8519
Epoch: 8/500... Training loss: 7.7582
Epoch: 8/500... Training loss: 7.7161
Epoch: 8/500... Training loss: 7.6434
Epoch: 8/500... Training loss: 7.6053
Epoch: 8/500... Training loss: 7.6815
Epoch: 8/500... Training loss: 7.8804
Epoch: 8/500... Training loss: 8.1125
Epoch: 9/500... Training loss: 7.0419
Epoch: 9/500... Training loss: 7.1811
Epoch: 9/500... Training loss: 7.3796
Epoch: 9/500... Training loss: 7.6158
Epoch: 9/500

Epoch: 15/500... Training loss: 2.6385
Epoch: 15/500... Training loss: 2.4950
Epoch: 15/500... Training loss: 2.1548
Epoch: 15/500... Training loss: 2.2765
Epoch: 15/500... Training loss: 2.4482
Epoch: 15/500... Training loss: 2.3945
Epoch: 15/500... Training loss: 2.3681
Epoch: 15/500... Training loss: 2.8531
Epoch: 15/500... Training loss: 2.6264
Epoch: 15/500... Training loss: 2.6192
Epoch: 15/500... Training loss: 2.2952
Epoch: 16/500... Training loss: 2.5871
Epoch: 16/500... Training loss: 2.7351
Epoch: 16/500... Training loss: 2.6989
Epoch: 16/500... Training loss: 2.7235
Epoch: 16/500... Training loss: 2.4095
Epoch: 16/500... Training loss: 2.6310
Epoch: 16/500... Training loss: 2.3611
Epoch: 16/500... Training loss: 2.4553
Epoch: 16/500... Training loss: 2.4286
Epoch: 16/500... Training loss: 2.4903
Epoch: 16/500... Training loss: 2.6429
Epoch: 16/500... Training loss: 2.5373
Epoch: 16/500... Training loss: 2.5653
Epoch: 16/500... Training loss: 2.3883
Epoch: 16/500... Training

Epoch: 22/500... Training loss: 1.1649
Epoch: 22/500... Training loss: 0.9848
Epoch: 23/500... Training loss: 1.6827
Epoch: 23/500... Training loss: 1.4174
Epoch: 23/500... Training loss: 1.3302
Epoch: 23/500... Training loss: 1.3530
Epoch: 23/500... Training loss: 1.0778
Epoch: 23/500... Training loss: 1.3316
Epoch: 23/500... Training loss: 1.1143
Epoch: 23/500... Training loss: 1.2549
Epoch: 23/500... Training loss: 1.1586
Epoch: 23/500... Training loss: 1.3425
Epoch: 23/500... Training loss: 1.5794
Epoch: 23/500... Training loss: 1.4959
Epoch: 23/500... Training loss: 1.1619
Epoch: 23/500... Training loss: 1.1716
Epoch: 23/500... Training loss: 1.3511
Epoch: 23/500... Training loss: 1.0093
Epoch: 23/500... Training loss: 1.1032
Epoch: 23/500... Training loss: 1.2904
Epoch: 23/500... Training loss: 1.1200
Epoch: 23/500... Training loss: 1.0834
Epoch: 23/500... Training loss: 1.4323
Epoch: 23/500... Training loss: 1.2709
Epoch: 23/500... Training loss: 0.9868
Epoch: 23/500... Training

Epoch: 30/500... Training loss: 0.9229
Epoch: 30/500... Training loss: 0.7921
Epoch: 30/500... Training loss: 0.7491
Epoch: 30/500... Training loss: 1.1599
Epoch: 30/500... Training loss: 1.1555
Epoch: 30/500... Training loss: 0.9829
Epoch: 30/500... Training loss: 0.6862
Epoch: 30/500... Training loss: 0.9526
Epoch: 30/500... Training loss: 0.5999
Epoch: 30/500... Training loss: 0.6860
Epoch: 30/500... Training loss: 0.8942
Epoch: 30/500... Training loss: 0.8559
Epoch: 30/500... Training loss: 0.6624
Epoch: 30/500... Training loss: 1.0038
Epoch: 30/500... Training loss: 0.8710
Epoch: 30/500... Training loss: 0.7223
Epoch: 30/500... Training loss: 0.7912
Epoch: 30/500... Training loss: 0.7149
Epoch: 30/500... Training loss: 0.7506
Epoch: 30/500... Training loss: 0.9519
Epoch: 30/500... Training loss: 0.8314
Epoch: 30/500... Training loss: 0.8548
Epoch: 30/500... Training loss: 0.7065
Epoch: 30/500... Training loss: 0.6613
Epoch: 31/500... Training loss: 1.1805
Epoch: 31/500... Training

...

Epoch: 468/500... Training loss: 0.0053
Epoch: 468/500... Training loss: 0.0146
Epoch: 468/500... Training loss: 0.0113
Epoch: 468/500... Training loss: 0.0012
Epoch: 468/500... Training loss: 0.0017
Epoch: 468/500... Training loss: 0.0852
Epoch: 468/500... Training loss: 0.0019
Epoch: 468/500... Training loss: 0.0028
Epoch: 468/500... Training loss: 0.0023
Epoch: 468/500... Training loss: 0.0040
Epoch: 468/500... Training loss: 0.0045
Epoch: 468/500... Training loss: 0.0012
Epoch: 468/500... Training loss: 0.0020
Epoch: 468/500... Training loss: 0.0002
Epoch: 468/500... Training loss: 0.0050
Epoch: 468/500... Training loss: 0.0008
Epoch: 468/500... Training loss: 0.0008
Epoch: 468/500... Training loss: 0.0032
Epoch: 468/500... Training loss: 0.0766
Epoch: 468/500... Training loss: 0.0007
Epoch: 468/500... Training loss: 0.0006
Epoch: 468/500... Training loss: 0.0140
Epoch: 468/500... Training loss: 0.0120
Epoch: 468/500... Training loss: 0.0044
Epoch: 468/500... Training loss: 0.0020


Epoch: 474/500... Training loss: 0.0019
Epoch: 474/500... Training loss: 0.0023
Epoch: 474/500... Training loss: 0.0051
Epoch: 474/500... Training loss: 0.0021
Epoch: 474/500... Training loss: 0.0010
Epoch: 474/500... Training loss: 0.0089
Epoch: 475/500... Training loss: 0.0018
Epoch: 475/500... Training loss: 0.0470
Epoch: 475/500... Training loss: 0.0019
Epoch: 475/500... Training loss: 0.0014
Epoch: 475/500... Training loss: 0.0051
Epoch: 475/500... Training loss: 0.0011
Epoch: 475/500... Training loss: 0.0061
Epoch: 475/500... Training loss: 0.0474
Epoch: 475/500... Training loss: 0.0018
Epoch: 475/500... Training loss: 0.0029
Epoch: 475/500... Training loss: 0.0075
Epoch: 475/500... Training loss: 0.0015
Epoch: 475/500... Training loss: 0.0093
Epoch: 475/500... Training loss: 0.0079
Epoch: 475/500... Training loss: 0.0099
Epoch: 475/500... Training loss: 0.0110
Epoch: 475/500... Training loss: 0.0131
Epoch: 475/500... Training loss: 0.0244
Epoch: 475/500... Training loss: 0.0058


Epoch: 481/500... Training loss: 0.0027
Epoch: 481/500... Training loss: 0.0005
Epoch: 481/500... Training loss: 0.0016
Epoch: 481/500... Training loss: 0.0186
Epoch: 481/500... Training loss: 0.0430
Epoch: 481/500... Training loss: 0.0082
Epoch: 481/500... Training loss: 0.0062
Epoch: 481/500... Training loss: 0.0054
Epoch: 481/500... Training loss: 0.0077
Epoch: 481/500... Training loss: 0.0062
Epoch: 481/500... Training loss: 0.0003
Epoch: 481/500... Training loss: 0.0007
Epoch: 481/500... Training loss: 0.0124
Epoch: 481/500... Training loss: 0.0051
Epoch: 481/500... Training loss: 0.0060
Epoch: 482/500... Training loss: 0.0017
Epoch: 482/500... Training loss: 0.0075
Epoch: 482/500... Training loss: 0.0009
Epoch: 482/500... Training loss: 0.0019
Epoch: 482/500... Training loss: 0.0044
Epoch: 482/500... Training loss: 0.0030
Epoch: 482/500... Training loss: 0.0109
Epoch: 482/500... Training loss: 0.0005
Epoch: 482/500... Training loss: 0.0103
Epoch: 482/500... Training loss: 0.0032


Epoch: 488/500... Training loss: 0.0007
Epoch: 488/500... Training loss: 0.0005
Epoch: 488/500... Training loss: 0.0019
Epoch: 488/500... Training loss: 0.0012
Epoch: 488/500... Training loss: 0.0066
Epoch: 488/500... Training loss: 0.0006
Epoch: 488/500... Training loss: 0.0007
Epoch: 488/500... Training loss: 0.0035
Epoch: 488/500... Training loss: 0.0029
Epoch: 488/500... Training loss: 0.0004
Epoch: 488/500... Training loss: 0.0041
Epoch: 488/500... Training loss: 0.0013
Epoch: 488/500... Training loss: 0.0011
Epoch: 488/500... Training loss: 0.0213
Epoch: 488/500... Training loss: 0.0037
Epoch: 488/500... Training loss: 0.0014
Epoch: 488/500... Training loss: 0.0035
Epoch: 488/500... Training loss: 0.0144
Epoch: 488/500... Training loss: 0.0014
Epoch: 488/500... Training loss: 0.0424
Epoch: 488/500... Training loss: 0.0315
Epoch: 488/500... Training loss: 0.0046
Epoch: 488/500... Training loss: 0.0016
Epoch: 489/500... Training loss: 0.0017
Epoch: 489/500... Training loss: 0.0018


Epoch: 494/500... Training loss: 0.0018
Epoch: 495/500... Training loss: 0.0410
Epoch: 495/500... Training loss: 0.0043
Epoch: 495/500... Training loss: 0.0046
Epoch: 495/500... Training loss: 0.0056
Epoch: 495/500... Training loss: 0.0017
Epoch: 495/500... Training loss: 0.0505
Epoch: 495/500... Training loss: 0.1107
Epoch: 495/500... Training loss: 0.0661
Epoch: 495/500... Training loss: 0.0041
Epoch: 495/500... Training loss: 0.0014
Epoch: 495/500... Training loss: 0.1481
Epoch: 495/500... Training loss: 0.0083
Epoch: 495/500... Training loss: 0.0325
Epoch: 495/500... Training loss: 0.0052
Epoch: 495/500... Training loss: 0.1095
Epoch: 495/500... Training loss: 0.0006
Epoch: 495/500... Training loss: 0.0011
Epoch: 495/500... Training loss: 0.0009
Epoch: 495/500... Training loss: 0.0306
Epoch: 495/500... Training loss: 0.0021
Epoch: 495/500... Training loss: 0.0020
Epoch: 495/500... Training loss: 0.0019
Epoch: 495/500... Training loss: 0.0069
Epoch: 495/500... Training loss: 0.0007


In [10]:

originals = ['adams twp', 'cranberry twp', 'middlesex twp', 'mars boro', 'allegheny twp', 
    'bruin boro', 'brady twp', 'slippery rock twp', 'buffalo twp', 'winfield twp', 'butler city', 
    'butler twp', 'penn twp', 'callery boro', 'center twp', 'clay twp', 'cherry twp', 'cherry valley boro', 
    'chicora boro', 'clearfield twp', 'donegal twp', 'summit twp', 'clinton twp', 'connoq boro', 'forward twp', 
    'connoq twp', 'lancaster twp', 'seven fields boro', 'fairview twp', 'east butler boro']


test_words = ['adam tp', 'cranbrry twp', 'middlesx twp', 'mars bro', 'alleghey twp', 
   'buin boro', 'brady twp', 'slippery rock twp', 'buffalo twp', 'winfield twp', 'butler city', 
   'butler twp', 'pen twp', 'callry boro', 'center twp', 'clay twp', 'cherry twp', 'cherry valley boro', 
   'chicora buro', 'clearyield tp', 'dolegal twp', 'sumit twp', 'clnon twp', 'connoq boro', 'forard twp', 
   'connoq twp', 'lanpaster twp', 'sevez fiels boro', 'fairview twp', 'east buter boro']

in_words = []
for word in test_words:
    X = np.zeros(vocab_size)
    for j in range(len(word)):
        if word[j] in vocab_to_id:
            X[vocab_to_id[word[j]]] += 1
            
    # normalize input sizes 
    X = ((X - X.min()) / (X.max() - X.min())) + 0.001
    in_words.append(X)
    
in_words = np.array(in_words)

reconstructed, compressed = sess.run([logits, encoded], feed_dict={inputs_: in_words})

out_words = []
for out, inp, org in zip(reconstructed, test_words, originals):
    best_match = np.argmax(out)
    
    out_words.append((id_to_address[labels_to_id[best_match]], inp, org))
    
        
for i in out_words:
    print(i)

('tampa', 'adam tp', 'adams twp')
('cranberry twp', 'cranbrry twp', 'cranberry twp')
('colley twp', 'middlesx twp', 'middlesex twp')
('cresson', 'mars bro', 'mars boro')
('whately', 'alleghey twp', 'allegheny twp')
('blossburg boro', 'buin boro', 'bruin boro')
('brady twp', 'brady twp', 'brady twp')
('slippery rock twp', 'slippery rock twp', 'slippery rock twp')
('west buffalo twp', 'buffalo twp', 'buffalo twp')
('winfield twp', 'winfield twp', 'winfield twp')
('butler city', 'butler city', 'butler city')
('butler twp', 'butler twp', 'butler twp')
('penn twp', 'pen twp', 'penn twp')
('cairo', 'callry boro', 'callery boro')
('center twp', 'center twp', 'center twp')
('clay twp', 'clay twp', 'clay twp')
('cherry twp', 'cherry twp', 'cherry twp')
('cherry valley', 'cherry valley boro', 'cherry valley boro')
('starrucca borough', 'chicora buro', 'chicora boro')
('ripley', 'clearyield tp', 'clearfield twp')
('ogle twp', 'dolegal twp', 'donegal twp')
('summittwp', 'sumit twp', 'summit twp')
